In [ ]:
# import and download all required packages 

In [ ]:
import nltk
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict 
import numpy as np
from statistics import variance
import nltk
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=eb4e267c8d9d1fb0bbe26c350f4465556608b01d63dfedc7780369592c223957
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer 
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
dataset = pd.read_csv("/content/gdrive/MyDrive/nlp/Clickbit_dataset/clickbait17-validation-170330/instances.csv")

In [ ]:
dataset.shape

(2459, 9)

In [ ]:
dataset.head(3)

,id,postTimestamp,postText,postMedia,targetTitle,targetDescription,targetKeywords,targetParagraphs,targetCaptions
0,608310377143799810,Tue Jun 09 16:31:10 +0000 2015,"[""Apple's iOS 9 'App thinning' feature will gi...",[],Apple gives back gigabytes: iOS 9 'app thinnin...,'App thinning' will be supported on Apple's iO...,"Apple,gives,gigabytes,iOS,9,app,thinning,featu...","[""Paying for a 64GB phone only to discover tha...","[""'App thinning' will be supported on Apple's ..."
1,609297109095972864,Fri Jun 12 09:52:05 +0000 2015,['RT @kenbrown12: Emerging market investors ar...,['media/609297109095972864.jpg'],Emerging Markets Suffer Largest Outflow in Sev...,Global investors have yanked $9.3 billion from...,"emerging market,emerging markets,em flows,em i...","['Emerging markets are out of favor.', 'Global...",['Stocks Fall as Investors Watch Central Banks...
2,609504474621612032,Fri Jun 12 23:36:05 +0000 2015,['U.S. Soccer should start answering tough que...,[],U.S. Soccer should start answering tough quest...,A U.S. Senator's scathing letter questioned U....,NaN,"[""WINNIPEG, Manitoba – The bubble U.S. Soccer ...",['US to vote for Ali in FIFA election and not ...


In [ ]:
text = dataset["postText"]

In [ ]:
title = dataset["targetTitle"]

In [ ]:
# Removing the emoji from text 

In [ ]:
import re
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
postText = dataset["postText"]
for i in range(len(postText)):
  t = remove_emoji(postText[i])
  t = t[2:-2] 
  postText[i] = t

<ipython-input-12-7c1c83d1c3e3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postText[i] = t


In [ ]:
postText[1]

'RT @kenbrown12: Emerging market investors are doing their best Monty Pythons--"Run away, run away"'

In [ ]:
title[2]

'U.S. Soccer should start answering tough questions about Hope Solo'

In [ ]:
postText[2]

'U.S. Soccer should start answering tough questions about Hope Solo, @eric_adelson writes.'

In [ ]:
title[1]

'Emerging Markets Suffer Largest Outflow in Seven Years'

In [ ]:
postText[1]

'RT @kenbrown12: Emerging market investors are doing their best Monty Pythons--"Run away, run away"'

In [ ]:
# Finding the similarity between post texts and titles using cosine similarity. Sentences BERT encoder used to encode the titles and post texts

In [ ]:
def sim_title_pos(post,ttl):
  l1 = list(post)
  l2 = list(ttl)
  l =[]
  for i in range(len(post)):
    if len(l1[i])==0 or len(l2[i])==0:
      l.append(0)
      continue
    arr0 = model.encode(l1[i])
    arr1 = model.encode(l2[i])
    arr = cosine_similarity([arr0],[arr1])
    l.append(arr)
  return l
l_=sim_title_pos(postText,title)

In [ ]:
arr1 = np.array(l_)

In [ ]:
arr1

array([[[0.95827156]],

       [[0.34847686]],

       [[0.9461709 ]],

       ...,

       [[0.92362374]],

       [[0.61657435]],

       [[0.8596171 ]]], dtype=float32)

In [ ]:
arr_faltten = arr1.flatten()

In [ ]:
arr_faltten

array([0.95827156, 0.34847686, 0.9461709 , ..., 0.92362374, 0.61657435,
       0.8596171 ], dtype=float32)

In [ ]:
type(pd.DataFrame(arr_faltten))

pandas.core.frame.DataFrame

In [ ]:
df1 = pd.DataFrame()
df1["id"] = dataset["id"]
df1["sim between postText and Title"] =  pd.DataFrame(arr_faltten)


In [ ]:
k = dataset["targetKeywords"]

In [ ]:
# Find the number of common words b/w PostText & Keyword

In [ ]:
def Match(list1,list2):
  l = []
  for i in range(len(list2)):
    s1 = list2[i].split(",")
    s2 = list1[i][0].lower()
    cnt =0
    for item in s1:
       if s2.find(item.lower())!=-1:
         cnt=cnt+1
    if(cnt==0):
    l.append(cnt)
  df1["Common b/w PostText & Keyword"] = pd.DataFrame(l)


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/nlp/Model/csv/postText.csv")

In [ ]:
# Add the features with existing features 

In [ ]:
data = pd.merge(df,df1,on='id', how='inner')

In [ ]:
data.shape

(2459, 58)

In [ ]:
data.to_csv("/content/gdrive/MyDrive/nlp/Model/csv/postText.csv", index = False)